In [ ]:
# %% [markdown]
# # Cell 1: Configuration, API Interaction, and Data Download/Update
#
# This cell handles:
# 1. Loading configuration (API keys) from `.env`.
# 2. Defining necessary endpoints and constants.
# 3. Authenticating with the IGDB API via Twitch.
# 4. Checking the status of local data dumps against the latest available dumps.
# 5. Downloading or updating any required dumps to the `DATA_DUMP_DIR`.
#
# **Output:** Populated `dump_file_paths` dictionary mapping endpoint names to their local file paths, ready for Cell 2.

# %%
import os
import requests
import time
import math
import json
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime, timezone
import chime
# --- Configuration ---
load_dotenv() # Charge les variables depuis le fichier .env
print(f"Environnement .env chargé: {'OUI' if load_dotenv() else 'NON'}")


# Clés API depuis .env
CLIENT_ID = os.getenv("IGDB_CLIENT_ID")
CLIENT_SECRET = os.getenv("IGDB_CLIENT_SECRET")
print(f"Client ID chargé: {'Oui' if CLIENT_ID else 'NON'}")
print(f"Client Secret chargé: {'Oui' if CLIENT_SECRET else 'NON'}")


# Chemins et URLs
DATA_DUMP_DIR = Path('igdb_datadumps') 
TOKEN_URL = 'https://id.twitch.tv/oauth2/token'
API_BASE_URL = 'https://api.igdb.com/v4'

# Crée le répertoire de destination s'il n'existe pas
DATA_DUMP_DIR.mkdir(parents=True, exist_ok=True)
print(f"Répertoire des dumps: {DATA_DUMP_DIR.resolve()}")


# Endpoints nécessaires pour la table `public.games` + lookups
# Inclut TOUS les endpoints dont les CSV sont nécessaires pour le traitement en Cellule 2
REQUIRED_ENDPOINTS = [
    'games', 'genres', 'platforms', 'themes', 'keywords', 'game_modes',
    'player_perspectives', 'collections', 'franchises', 'game_types',
    'game_statuses', 'involved_companies', 'companies', 'external_games',
    'popularity_primitives', 'covers', 'artworks',
    'screenshots', 'alternative_names', 'external_game_sources',
    'websites', 'website_types',
    # Ajouts pour les mappings nécessaires
    'game_localizations',
    'language_supports',
    'languages', # Nécessaire pour mapper les noms/locales de language_supports
    'multiplayer_modes',
    'age_ratings',
    'release_dates', # Absolument nécessaire pour le calcul de release_date min
    'tags',
    # 'popularity_types', # Peut être utile pour vérifier les IDs de POPULARITY_TYPE_IDS
    # 'age_rating_content_descriptions', # Si besoin de plus de détails sur age_ratings
]
print(f"Endpoints requis: {len(REQUIRED_ENDPOINTS)}")


# --- Fonctions API et Téléchargement (Robustes) ---

def get_access_token(client_id, client_secret):
    """Récupère un token d'accès Twitch/IGDB."""
    print("Récupération du token d'accès Twitch...")
    if not client_id or not client_secret:
        print("ERREUR: CLIENT_ID ou CLIENT_SECRET manquant.")
        return None
    try:
        response = requests.post(TOKEN_URL, params={
            'client_id': client_id, 'client_secret': client_secret,
            'grant_type': 'client_credentials'
        }, timeout=15) # Timeout légèrement augmenté
        response.raise_for_status() # Lève une exception pour les codes 4xx/5xx
        token_data = response.json()
        access_token = token_data.get('access_token')
        if access_token:
            print("Token obtenu avec succès.")
            return access_token
        else:
            print("ERREUR: Token non trouvé dans la réponse.", token_data)
            return None
    except requests.exceptions.Timeout:
        print("ERREUR: Timeout lors de la demande de token.")
    except requests.exceptions.RequestException as e:
        error_message = f"Erreur lors de la récupération du token : {e}"
        if hasattr(e, 'response') and e.response is not None:
            try:
                 error_message += f"\nRéponse serveur ({e.response.status_code}): {e.response.text}"
            except Exception:
                 error_message += f"\nRéponse serveur ({e.response.status_code}): [Impossible de lire le corps]"
        print(error_message)
    return None

def get_api_headers(token):
    """Construit les headers nécessaires pour les appels API IGDB."""
    if not token or not CLIENT_ID:
        print("ERREUR: Token ou Client ID manquant pour créer les headers.")
        return None
    return {
        'Client-ID': CLIENT_ID,
        'Authorization': f'Bearer {token}',
        'Accept': 'application/json'
    }

def get_available_dumps(headers):
    """Récupère la liste des data dumps disponibles depuis l'API."""
    print("Récupération de la liste des data dumps disponibles...")
    if not headers: return None
    try:
        response = requests.get(f'{API_BASE_URL}/dumps', headers=headers, timeout=30)
        response.raise_for_status()
        dumps_list = response.json()
        print(f"Liste des dumps obtenue ({len(dumps_list)} dumps trouvés).")
        # Convertit la liste en dictionnaire {endpoint_name: dump_info}
        return {dump['endpoint']: dump for dump in dumps_list if 'endpoint' in dump}
    except requests.exceptions.Timeout:
        print("ERREUR: Timeout lors de la récupération de la liste des dumps.")
    except requests.exceptions.RequestException as e:
        print(f"ERREUR lors de la récupération de la liste des dumps : {e}", f"Réponse: {e.response.text}" if hasattr(e, 'response') and e.response else "")
    return None

def get_dump_download_url(endpoint, headers):
    """Obtient l'URL S3 pour télécharger un dump spécifique."""
    print(f"  Obtention URL de téléchargement pour '{endpoint}'...")
    if not headers: return None
    url = f'{API_BASE_URL}/dumps/{endpoint}'
    try:
        response = requests.get(url, headers=headers, timeout=20) # Timeout un peu plus long
        response.raise_for_status()
        s3_url = response.json().get('s3_url')
        if s3_url:
            print(f"  URL S3 obtenue pour '{endpoint}'.")
            return s3_url
        else:
            print(f"  ERREUR: URL S3 non trouvée dans la réponse pour '{endpoint}'.")
            return None
    except requests.exceptions.Timeout:
        print(f"  ERREUR: Timeout lors de l'obtention de l'URL du dump '{endpoint}'.")
    except requests.exceptions.RequestException as e:
        print(f"  ERREUR lors de l'obtention de l'URL du dump '{endpoint}' ({url}) : {e}", f"Réponse: {e.response.text}" if hasattr(e, 'response') and e.response else "")
    return None

def download_file(url, dest_path):
    """Télécharge un fichier depuis une URL vers un chemin local avec barre de progression."""
    print(f"  Téléchargement de {dest_path.name} depuis {url[:50]}...")
    try:
        with requests.get(url, stream=True, timeout=600) as r: # Timeout long pour gros fichiers
            r.raise_for_status()
            total_size = int(r.headers.get('content-length', 0))
            downloaded_size = 0
            last_print_time = time.time()
            print(f"  Taille totale: {total_size/1024/1024:.1f} Mo" if total_size > 0 else "  Taille inconnue.")
            with open(dest_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192 * 16): # Chunk de 128Ko
                    if chunk: # Filtre les keep-alive chunks
                        f.write(chunk)
                        downloaded_size += len(chunk)
                        now = time.time()
                        if total_size > 0 and (now - last_print_time > 1): # MAJ toutes les secondes
                            percent = (downloaded_size / total_size) * 100
                            print(f"    -> {downloaded_size/1024/1024:.1f} / {total_size/1024/1024:.1f} Mo ({percent:.1f}%)", end='\r')
                            last_print_time = now
            # Assurer que la ligne de progression est effacée
            print(' ' * 80, end='\r')
            print(f"  {dest_path.name} téléchargé avec succès.")
        return True
    except requests.exceptions.Timeout:
        print(f"\n  ERREUR: Timeout pendant le téléchargement de {dest_path.name}.")
    except requests.exceptions.RequestException as e:
        print(f"\n  ERREUR pendant le téléchargement de {dest_path.name} : {e}")
    except Exception as e:
        print(f"\n  ERREUR inconnue pendant le téléchargement de {dest_path.name} : {e}")

    # Nettoyage en cas d'échec
    if dest_path.exists():
        try:
            dest_path.unlink()
            print(f"  Fichier incomplet {dest_path.name} supprimé.")
        except OSError as e_del:
            print(f"  ERREUR lors de la suppression du fichier incomplet {dest_path.name}: {e_del}")
    return False

def check_and_download_dump(endpoint, dump_info, headers, local_dir):
    """Vérifie si un dump local est à jour et le télécharge si nécessaire."""
    print(f"\n--- Vérification Endpoint: {endpoint} ---")
    if not dump_info:
        print(f"ATTENTION: Aucune information de dump trouvée pour '{endpoint}' via l'API. Ignoré.")
        return None # Retourne None si l'endpoint n'existe pas côté API

    remote_timestamp = dump_info.get('updated_at')
    file_name = dump_info.get('file_name')

    if not remote_timestamp or not file_name:
        print(f"ERREUR: Données de dump incomplètes pour '{endpoint}' (timestamp ou nom de fichier manquant).")
        return None # Ne peut pas traiter sans ces infos

    local_file_path = local_dir / file_name
    needs_download = False

    if not local_file_path.exists():
        print(f"Fichier local '{file_name}' non trouvé.")
        needs_download = True
    else:
        try:
            local_timestamp = local_file_path.stat().st_mtime
            # Comparer les timestamps UNIX directement
            # +1 pour une petite marge (évite re-téléchargement si différence < 1s)
            if remote_timestamp > local_timestamp + 1:
                remote_dt = datetime.fromtimestamp(remote_timestamp, timezone.utc)
                local_dt = datetime.fromtimestamp(local_timestamp, timezone.utc)
                print(f"Dump distant '{endpoint}' plus récent.")
                print(f"  Distant: {remote_dt.strftime('%Y-%m-%d %H:%M:%S %Z')}")
                print(f"  Local:   {local_dt.strftime('%Y-%m-%d %H:%M:%S %Z')}")
                needs_download = True
            else:
                print(f"Fichier local '{file_name}' est à jour.")
                return local_file_path # Retourne le chemin si à jour
        except Exception as e:
            print(f"ERREUR lors de la vérification du fichier local {file_name}: {e}")
            print("Téléchargement par précaution.")
            needs_download = True

    if needs_download:
        s3_url = get_dump_download_url(endpoint, headers)
        if s3_url:
            # Supprimer l'ancien fichier avant de télécharger le nouveau
            if local_file_path.exists():
                print(f"  Suppression de l'ancien fichier {local_file_path.name}...")
                try:
                    local_file_path.unlink()
                except OSError as e_del:
                    print(f"  ERREUR lors de la suppression de l'ancien {local_file_path.name}: {e_del}")
                    # On tente quand même le téléchargement, il écrasera peut-être

            # Lancer le téléchargement
            if download_file(s3_url, local_file_path):
                # Vérifier la taille après téléchargement si possible (basique)
                if local_file_path.stat().st_size > 0:
                     print(f"Téléchargement de '{endpoint}' terminé.")
                     return local_file_path # Succès
                else:
                     print(f"ERREUR: Fichier '{endpoint}' téléchargé mais taille nulle.")
                     if local_file_path.exists(): local_file_path.unlink() # Supprimer fichier vide
                     return None # Echec
            else:
                print(f"Echec du téléchargement pour '{endpoint}'.")
                return None # Echec
        else:
            print(f"Impossible d'obtenir l'URL de téléchargement pour '{endpoint}'.")
            return None # Echec

    # Si on arrive ici, c'est que needs_download était False
    return local_file_path

# --- Exécution du téléchargement ---
start_time_cell1 = time.time()
print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] == Début Cellule 1: Téléchargement/Vérification des Dumps ==")

# Vérification des clés API
if not CLIENT_ID or not CLIENT_SECRET:
    print("###########################################################################")
    print(" ERREUR CRITIQUE: Clés IGDB (CLIENT_ID, CLIENT_SECRET) non trouvées.")
    print(" Vérifiez votre fichier .env ou les variables d'environnement.")
    print(" Le script ne peut pas continuer sans les clés API.")
    print("###########################################################################")
    # Dans Jupyter, on ne peut pas utiliser exit(), mais on peut lever une exception
    raise ValueError("Clés API IGDB manquantes. Arrêt du script.")

# Obtention du token
access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)
if not access_token:
    raise ValueError("Echec de l'obtention du token d'accès Twitch/IGDB. Vérifiez vos clés API et la connectivité. Arrêt du script.")

# Préparation des headers
api_headers = get_api_headers(access_token)
if not api_headers:
    raise ValueError("Echec de la préparation des headers API. Arrêt du script.")

# Récupération de la liste des dumps disponibles
available_dumps_info = get_available_dumps(api_headers)
if not available_dumps_info:
    raise ConnectionError("Echec de la récupération de la liste des dumps disponibles depuis IGDB. Vérifiez la connexion et le statut de l'API. Arrêt du script.")

# Vérification et téléchargement des dumps requis
print(f"\n[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Vérification/Téléchargement des {len(REQUIRED_ENDPOINTS)} dumps requis...")
download_results = {}
dump_file_paths = {} # Sera utilisé par la cellule 2
all_required_dumps_ready = True
missing_critical_dumps = []

for endpoint in REQUIRED_ENDPOINTS:
    t0_endpoint = time.time()
    dump_info = available_dumps_info.get(endpoint)
    file_path = check_and_download_dump(endpoint, dump_info, api_headers, DATA_DUMP_DIR)

    if file_path and file_path.exists() and file_path.stat().st_size > 0:
        download_results[endpoint] = True
        dump_file_paths[endpoint] = file_path # Stocker le chemin
        print(f"--- Fin {endpoint} (OK) ({(time.time() - t0_endpoint):.2f}s) ---")
    else:
        download_results[endpoint] = False
        print(f"*** ECHEC ou fichier invalide pour l'endpoint requis '{endpoint}' ***")
        all_required_dumps_ready = False
        # Marquer comme critique si c'est 'games' ou un lookup essentiel
        if endpoint in ['games', 'platforms', 'release_dates', 'companies', 'involved_companies']:
             missing_critical_dumps.append(endpoint)
        print(f"--- Fin {endpoint} (ECHEC) ({(time.time() - t0_endpoint):.2f}s) ---")


# Bilan final de la Cellule 1
print(f"\n[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] == Bilan Cellule 1 ==")
successful_downloads = sum(1 for ready in download_results.values() if ready)
failed_downloads = len(REQUIRED_ENDPOINTS) - successful_downloads
print(f"Endpoints traités: {len(REQUIRED_ENDPOINTS)}")
print(f"  Succès: {successful_downloads}")
print(f"  Échecs: {failed_downloads}")
if not all_required_dumps_ready:
    print("\n###########################################################################")
    print(" ATTENTION: Certains dumps requis n'ont pas pu être préparés.")
    print(f" Endpoints en échec: {[ep for ep, ready in download_results.items() if not ready]}")
    if missing_critical_dumps:
        print(f" Dumps CRITIQUES manquants: {missing_critical_dumps}")
        print(" L'exécution de la Cellule 2 risque d'échouer ou de produire des résultats incorrects.")
        # Consider raising an error here if critical files are missing
        # raise FileNotFoundError(f"Dumps critiques manquants: {missing_critical_dumps}. Arrêt.")
    else:
        print(" La Cellule 2 tentera de continuer avec les dumps disponibles, mais les résultats peuvent être incomplets.")
    print("###########################################################################")
else:
    print("\nTous les dumps requis sont prêts.")

print(f"Dumps prêts et leurs chemins:")
for endpoint, path in dump_file_paths.items():
    print(f"  - {endpoint}: {path}")

total_time_cell1 = time.time() - start_time_cell1
print(f"\n[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] == Fin Cellule 1 (Temps total: {total_time_cell1:.2f}s) ==")

# Nettoyage des variables non nécessaires pour la cellule suivante (optionnel)
# del access_token, api_headers, available_dumps_info, download_results

Environnement .env chargé: OUI
Client ID chargé: Oui
Client Secret chargé: Oui
Répertoire des dumps: F:\Plish\Python\IGDB\igdb_datadumps
Endpoints requis: 29
[2025-06-16 13:27:02] == Début Cellule 1: Téléchargement/Vérification des Dumps ==
Récupération du token d'accès Twitch...
Token obtenu avec succès.
Récupération de la liste des data dumps disponibles...
Liste des dumps obtenue (68 dumps trouvés).

[2025-06-16 13:27:03] Vérification/Téléchargement des 29 dumps requis...

--- Vérification Endpoint: games ---
Fichier local '1750053600_games.csv' non trouvé.
  Obtention URL de téléchargement pour 'games'...
  URL S3 obtenue pour 'games'.
  Téléchargement de 1750053600_games.csv depuis https://api3000-base-apidatadump-1qytdzl2alob3.s3....
  Taille totale: 243.3 Mo
  1750053600_games.csv téléchargé avec succès.                                  
Téléchargement de 'games' terminé.
--- Fin games (OK) (39.94s) ---

--- Vérification Endpoint: genres ---
Fichier local '1750053600_genres.csv'

In [2]:
# %% [markdown]
# # Cell 3: Parser les dumps IGDB et générer le dataset de jeux (PostgreSQL Array Format - Image ID Fix)

# %%
import os
import json
import pandas as pd
import csv
import re
from slugify import slugify
import ast
from datetime import datetime
import logging
from pathlib import Path
import gzip
import time # Import time for potential date parsing

# Configuration du logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()


# --- PostgreSQL Array Formatting Function ---
def format_list_for_postgres_array(data_list, is_numeric=False):
    """
    Formats a Python list into a PostgreSQL array literal string.
    Handles quoting for text arrays as required by COPY.

    Args:
        data_list: The Python list/value to format.
        is_numeric: Set to True if the target column is integer[] or similar numeric array.

    Returns:
        A string formatted for PostgreSQL COPY (e.g., '{item1,item2,"item 3"}')
        Returns '' if data_list represents a NULL array (None, NaN).
        Returns '{}' for an empty list.
    """
    # 1. Check for Python None explicitly
    if data_list is None:
        return '' # Represent NULL array column for COPY

    # 2. Check for scalar NaN (common Pandas representation for missing)
    if isinstance(data_list, (float, int)) and pd.isna(data_list):
         return '' # Represent NULL array column for COPY

    # 3. If it's not None or scalar NaN, THEN check if it's a list/set/tuple
    if not isinstance(data_list, (list, set, tuple)):
         logger.warning(f"Input to format_list_for_postgres_array was not None/NaN or list/set/tuple: {type(data_list)}. Treating as empty array.")
         return '{}' # Empty array literal

    # Now we know data_list is a list, set, or tuple
    if not data_list:
        return '{}' # Empty array literal

    # Convert set/tuple to list for consistent processing
    if not isinstance(data_list, list):
        data_list = list(data_list)

    formatted_items = []
    for item in data_list:
        if item is None or pd.isna(item): # Check item-level NaN here
            formatted_items.append('NULL')
            continue

        item_str = str(item)

        if is_numeric:
             if item_str.strip().lower() == 'nan':
                 formatted_items.append('NULL')
             else:
                 try:
                     float(item_str) # Check if convertible
                     formatted_items.append(item_str.strip())
                 except ValueError:
                     logger.warning(f"Invalid numeric item '{item_str}' in list {data_list}. Appending NULL.")
                     formatted_items.append('NULL')
        else:
            # For text types, quote non-NULL elements.
            item_str_escaped = item_str.replace('\\', '\\\\').replace('"', '""')
            formatted_items.append(f'"{item_str_escaped}"')

    return '{' + ','.join(formatted_items) + '}'


# --- Image URL Construction Function ---
def get_igdb_image_url(image_id, size='screenshot_med'):
    """Constructs an IGDB image URL from an image_id and size."""
    if image_id and pd.notna(image_id) and image_id != '':
        # Ensure image_id doesn't already contain path separators
        image_id_cleaned = str(image_id).split('/')[-1]
        return f"https://images.igdb.com/igdb/image/upload/t_{size}/{image_id_cleaned}.jpg"
    return None # Return None if no valid image_id


# --- Other Helper Functions ---
def parse_json_list(data):
    # ... (Keep existing robust code) ...
    if data is None: return []
    if isinstance(data, list): return data
    if isinstance(data, str):
        if pd.isna(data) or data == '' or data == '[]' or data == '{}': return []
        try:
            cleaned_str = data.replace("'", '"')
            return json.loads(cleaned_str)
        except json.JSONDecodeError:
            try:
                evaluated_data = ast.literal_eval(data)
                if isinstance(evaluated_data, (list, tuple, set)): return list(evaluated_data)
                elif evaluated_data is not None: return [evaluated_data]
                else: return []
            except (ValueError, SyntaxError, TypeError) as e:
                 if isinstance(data, str) and ',' in data and not any(c in data for c in '[]{}'):
                      try: return [item.strip() for item in data.split(',') if item.strip()]
                      except Exception as split_e: logger.warning(f"Failed split fallback: {data}, Error: {split_e}"); return []
                 else: logger.warning(f"ast fallback failed: {data}, Error: {e}"); return []
        except Exception as e: logger.warning(f"Unexpected parse error: {data}, Error: {e}"); return []
    elif isinstance(data, (set, tuple)): return list(data)
    elif pd.notna(data): logger.debug(f"parse_json_list received non-list/str: {type(data)}"); return [data] # Wrap single items
    return []

def get_slug_list(obj_list):
    # ... (Keep existing code) ...
    slugs = []
    if not isinstance(obj_list, list): return []
    for obj in obj_list:
        name_to_slugify = None
        if isinstance(obj, dict) and 'name' in obj: name_to_slugify = obj.get('name')
        elif isinstance(obj, str): name_to_slugify = obj
        if name_to_slugify: slugs.append(slugify(str(name_to_slugify)))
    return slugs

def parse_id_list(id_list_data, ref_table=None):
    # ... (Keep existing robust code returning Python list) ...
    if id_list_data is None or pd.isna(id_list_data) or id_list_data == '': return []
    ids = []
    try:
        evaluated_data = None
        if isinstance(id_list_data, (list, set, tuple)): evaluated_data = list(id_list_data)
        elif isinstance(id_list_data, str):
             if id_list_data.strip() in ('[]', '{}', ''): return []
             try: evaluated_data = ast.literal_eval(id_list_data)
             except (ValueError, SyntaxError):
                 if ',' in id_list_data and not any(c in id_list_data for c in '[]{}'):
                     try:
                         items = [x.strip() for x in id_list_data.split(',') if x.strip()]
                         try: evaluated_data = [int(i) for i in items]
                         except ValueError: evaluated_data = items
                     except Exception: return []
                 else: return []
        elif pd.notna(id_list_data): evaluated_data = [id_list_data] # Wrap single items
        else: return []

        if isinstance(evaluated_data, (list, set, tuple)): ids = list(evaluated_data)
        elif evaluated_data is not None: ids = [evaluated_data]
        else: return []

        if ref_table is None:
            cleaned_ids = []
            for item in ids:
                if isinstance(item, (int, float)): cleaned_ids.append(item)
                elif isinstance(item, str) and item.isdigit(): cleaned_ids.append(int(item))
                elif isinstance(item, str): cleaned_ids.append(item)
            return cleaned_ids

        result = []
        for id_val in ids:
            resolved_name = None
            try: # Try lookup with original type, int, and str versions of the key
                 resolved_name = ref_table.get(id_val)
                 if resolved_name is None and isinstance(id_val, str) and id_val.isdigit(): resolved_name = ref_table.get(int(id_val))
                 elif resolved_name is None and isinstance(id_val, int): resolved_name = ref_table.get(str(id_val))
            except TypeError: pass # Ignore if key type is totally incompatible (e.g., list)
            result.append({"id": id_val, "name": resolved_name}) # Always add entry
        return result
    except Exception as e:
        logger.error(f"General error parsing ID list: {id_list_data}, Error: {e}", exc_info=True)
        return []

# %%
# --- Data Loading and Reference Table Creation ---

def get_dump_paths():
    # ... (Keep existing code) ...
    logger.info("Récupération des chemins des dumps IGDB depuis dump_file_paths...")
    dump_paths = {}
    if 'dump_file_paths' not in globals(): logger.error("Variable globale 'dump_file_paths' non trouvée."); return {}
    for endpoint, path in dump_file_paths.items():
        if not isinstance(path, (str, Path)): logger.warning(f"Chemin invalide pour {endpoint}: {path}. Ignoré."); continue
        dump_paths[endpoint] = Path(path); logger.info(f"Dump trouvé pour {endpoint}: {dump_paths[endpoint]}")
    return dump_paths

def load_dumps(dump_paths):
    # ... (Keep existing code) ...
    logger.info("Chargement des dumps IGDB en DataFrames...")
    dfs = {}
    if not dump_paths: logger.error("Aucun chemin de dump valide fourni."); return dfs
    for dump_type, path in dump_paths.items():
        try:
            logger.info(f"Chargement de {dump_type} depuis {path}...")
            if not path.exists(): logger.warning(f"Le fichier n'existe pas: {path}"); continue
            read_func = gzip.open if path.suffix == '.gz' else open
            encodings_to_try = ['utf-8', 'latin-1']
            df = None
            for enc in encodings_to_try:
                try:
                    with read_func(path, 'rt', encoding=enc) as f: df = pd.read_csv(f, low_memory=False); break
                except UnicodeDecodeError: continue
                except Exception as read_e: logger.error(f"Erreur lecture {dump_type} ({enc}): {read_e}"); break # Other errors
            if df is None: raise ValueError(f"Impossible de lire {path} avec les encodages testés.")
            dfs[dump_type] = df; logger.info(f"Chargé {dump_type}: {len(df)} entrées")
        except Exception as e: logger.error(f"Erreur chargement {dump_type} ({path}): {e}", exc_info=True); dfs[dump_type] = pd.DataFrame()
    return dfs

def create_reference_tables(dfs):
    logger.info("Création des tables de référence...")
    ref_tables = {}

    def create_simple_ref(table_name, id_col='id', val_col='name'):
        # ... (Keep existing code - ensure id_col keys are appropriate type) ...
        if table_name in dfs and not dfs[table_name].empty and id_col in dfs[table_name].columns and val_col in dfs[table_name].columns:
            df_ref = dfs[table_name][[id_col, val_col]].dropna(subset=[id_col])
            try: df_ref[id_col] = df_ref[id_col].astype(int); logger.debug(f"Converted {id_col} to int for {table_name}.")
            except (ValueError, TypeError): logger.debug(f"Keeping {id_col} as object/str for {table_name}."); df_ref[id_col] = df_ref[id_col].astype(str)
            df_ref = df_ref.drop_duplicates(subset=[id_col], keep='first')
            ref_tables[table_name] = dict(zip(df_ref[id_col], df_ref[val_col]))
            logger.info(f"Table ref '{table_name}' créée: {len(ref_tables[table_name])} entrées")
        else: # Log missing data/cols
            missing = [c for c in [id_col, val_col] if c not in dfs.get(table_name, pd.DataFrame()).columns] if table_name in dfs else []
            if table_name not in dfs or dfs[table_name].empty: logger.warning(f"Données manquantes/vides pour '{table_name}'")
            elif missing: logger.warning(f"Colonnes manquantes pour '{table_name}': {missing}")
            ref_tables[table_name] = {}

    create_simple_ref('genres'); create_simple_ref('platforms'); create_simple_ref('themes')
    create_simple_ref('keywords'); create_simple_ref('game_modes'); create_simple_ref('player_perspectives')
    create_simple_ref('collections'); create_simple_ref('franchises'); create_simple_ref('companies')
    create_simple_ref('game_types', val_col='type'); create_simple_ref('game_statuses', val_col='status')

    # Involved Companies (Keep existing code)
    # ... (stores dicts with company ID and roles per game ID) ...
    if 'involved_companies' in dfs and not dfs['involved_companies'].empty:
        req_cols = ['id', 'game', 'company', 'developer', 'publisher']
        if all(c in dfs['involved_companies'].columns for c in req_cols):
            inv_comps = {}
            for row in dfs['involved_companies'].itertuples(index=False):
                game_id = getattr(row, 'game', None); company = getattr(row, 'company', None)
                if pd.notna(game_id) and pd.notna(company):
                    try: game_id = int(game_id)
                    except: continue
                    if game_id not in inv_comps: inv_comps[game_id] = []
                    inv_info = {
                        'involved_company_id': int(getattr(row, 'id')) if pd.notna(getattr(row, 'id', None)) else None,
                        'company': int(company) if pd.notna(company) else None,
                        'developer': bool(getattr(row, 'developer', False)),
                        'publisher': bool(getattr(row, 'publisher', False)) }
                    inv_comps[game_id].append(inv_info)
            ref_tables['involved_companies'] = inv_comps
            logger.info(f"Table ref 'involved_companies' créée pour {len(inv_comps)} jeux")
        else: logger.warning(f"Colonnes manquantes pour 'involved_companies': {[c for c in req_cols if c not in dfs['involved_companies'].columns]}"); ref_tables['involved_companies'] = {}
    else: logger.warning("Données manquantes/vides pour 'involved_companies'"); ref_tables['involved_companies'] = {}

    # External Games (Keep existing code)
    # ... (stores dict with steam, psn, xbox IDs per game ID) ...
    if 'external_games' in dfs and not dfs['external_games'].empty:
        req_cols = ['game', 'category', 'uid']
        if all(c in dfs['external_games'].columns for c in req_cols):
            ext_games = {}
            for row in dfs['external_games'].itertuples(index=False):
                game_id = getattr(row, 'game', None); category = getattr(row, 'category', 0); uid = getattr(row, 'uid', '')
                if pd.notna(game_id):
                    try: game_id = int(game_id)
                    except: continue
                    if game_id not in ext_games: ext_games[game_id] = {'steam': '', 'psn': '', 'xbox': ''}
                    if pd.notna(uid) and uid != '':
                        uid_str = str(uid)
                        if category == 1: ext_games[game_id]['steam'] = uid_str
                        elif category == 36: ext_games[game_id]['psn'] = uid_str
                        elif category == 31: ext_games[game_id]['xbox'] = uid_str
            ref_tables['external_games'] = ext_games
            logger.info(f"Table ref 'external_games' créée pour {len(ext_games)} jeux")
        else: logger.warning(f"Colonnes manquantes pour 'external_games': {[c for c in req_cols if c not in dfs['external_games'].columns]}"); ref_tables['external_games'] = {}
    else: logger.warning("Données manquantes/vides pour 'external_games'"); ref_tables['external_games'] = {}


    # --- MODIFIED Media Tables (Store IMAGE IDs, not URLs) ---
    media_types_config = {
        # Focus on getting image_id
        'covers': {'id_col': 'image_id'},
        'artworks': {'id_col': 'image_id'},
        'screenshots': {'id_col': 'image_id'},
        # Websites still need URL
        'websites': {'url_col': 'url', 'id_col': 'id', 'extra_cols': ['category']}
    }

    for media_type, config in media_types_config.items():
        if media_type in dfs and not dfs[media_type].empty and 'game' in dfs[media_type].columns:
            media_dict = {}
            df_media = dfs[media_type].dropna(subset=['game'])

            # Check if essential column exists
            required_col = config.get('id_col') if media_type != 'websites' else config.get('url_col')
            if not required_col or required_col not in df_media.columns:
                logger.warning(f"Colonne requise '{required_col}' manquante pour '{media_type}'. Table de référence ignorée.")
                ref_tables[media_type] = {}
                continue

            for row in df_media.itertuples(index=False):
                try: game_id = int(getattr(row, 'game'))
                except: continue

                if game_id not in media_dict: media_dict[game_id] = []

                item_data = None
                if media_type == 'websites':
                    url = getattr(row, config['url_col'], None)
                    if pd.notna(url):
                        # Store dict with website details (as before)
                        website_info = {'url': str(url)}
                        for extra in config.get('extra_cols', []):
                            if hasattr(row, extra): val = getattr(row, extra); website_info[extra] = int(val) if pd.notna(val) and str(val).isdigit() else (str(val) if pd.notna(val) else None)
                        website_id = getattr(row, config.get('id_col', ''), None)
                        if pd.notna(website_id): website_info['website_id'] = int(website_id) if str(website_id).isdigit() else str(website_id)
                        item_data = website_info
                else: # Covers, Artworks, Screenshots: Store image_id
                    image_id = getattr(row, config['id_col'], None)
                    if pd.notna(image_id) and image_id != '':
                        # Store the image_id directly (cleaning just in case)
                        item_data = str(image_id).split('/')[-1] # Get final part if it looks like a path

                # Add unique data to the list for the game
                if item_data and item_data not in media_dict[game_id]:
                     media_dict[game_id].append(item_data)

            ref_tables[media_type] = media_dict
            logger.info(f"Table ref '{media_type}' créée pour {len(media_dict)} jeux (stocke {'image IDs' if media_type != 'websites' else 'website info'})")
        else: # Log reasons for skipping
            if media_type not in dfs or dfs[media_type].empty: logger.warning(f"Données manquantes/vides pour '{media_type}'")
            elif 'game' not in dfs[media_type].columns: logger.warning(f"Colonne 'game' manquante pour '{media_type}'")
            ref_tables[media_type] = {}

    return ref_tables


# %%
# --- Game Processing Function ---

def process_game(game, ref_tables):
    # ... (Keep existing structure) ...
    result = {}
    try: game_id = int(game['id'])
    except (ValueError, TypeError, KeyError): logger.error(f"Jeu ignoré: ID invalide/manquant: {game.get('id')}"); return None
    result['id_igdb'] = game_id

    # --- Basic Fields & External IDs ---
    result['slug'] = game.get('slug', '')
    result['title'] = game.get('name', '')
    ext_info = ref_tables.get('external_games', {}).get(game_id, {})
    steam_id = ext_info.get('steam', ''); result['steam_app_id'] = int(steam_id) if steam_id.isdigit() else pd.NA
    psn_id = ext_info.get('psn', ''); result['psn_app_id'] = int(psn_id) if psn_id.isdigit() else pd.NA
    result['xbox_app_id'] = ext_info.get('xbox', '')

    # --- Date/Timestamp Fields ---
    def parse_datetime_field(field_name):
        # ... (Keep existing robust code using pd.to_datetime) ...
        raw_value = game.get(field_name);
        if pd.isna(raw_value): return pd.NaT
        try: timestamp = int(float(raw_value)); return pd.to_datetime(timestamp, unit='s', utc=True) if timestamp > 0 else pd.NaT
        except (ValueError, TypeError): pass
        if isinstance(raw_value, str):
             try: dt = pd.to_datetime(raw_value, errors='coerce', utc=True); return dt
             except Exception: pass
        return pd.NaT
    result['release_date'] = parse_datetime_field('first_release_date')
    result['updated_at'] = datetime.now(timezone.utc)


    # --- MODIFIED Media Fields (Construct URL from ID) ---
    # cover: text (single URL, use a specific size like cover_big)
    cover_ids = ref_tables.get('covers', {}).get(game_id, [])
    first_cover_id = cover_ids[0] if cover_ids else None
    result['cover'] = get_igdb_image_url(first_cover_id, size='screenshot_med') or '' # Use empty string if no URL

    # artworks: text[] (list of URLs, use screenshot_med)
    artwork_ids = ref_tables.get('artworks', {}).get(game_id, [])
    artwork_urls = [get_igdb_image_url(img_id, size='screenshot_med') for img_id in artwork_ids]
    result['artworks'] = format_list_for_postgres_array([url for url in artwork_urls if url]) # Filter out None URLs

    # screenshots: text[] (list of URLs, use screenshot_med)
    screenshot_ids = ref_tables.get('screenshots', {}).get(game_id, [])
    screenshot_urls = [get_igdb_image_url(img_id, size='screenshot_med') for img_id in screenshot_ids]
    result['screenshots'] = format_list_for_postgres_array([url for url in screenshot_urls if url]) # Filter out None URLs

    # websites: text[] (extract URLs from dicts)
    websites_data = ref_tables.get('websites', {}).get(game_id, [])
    website_urls = [w['url'] for w in websites_data if isinstance(w, dict) and w.get('url')]
    result['websites'] = format_list_for_postgres_array(website_urls)


    # --- Array Fields (Integer) ---
    result['bundles'] = format_list_for_postgres_array(parse_id_list(game.get('bundles')), is_numeric=True)
    result['game_localizations'] = format_list_for_postgres_array(parse_id_list(game.get('game_localizations')), is_numeric=True)
    result['release_dates'] = format_list_for_postgres_array(parse_id_list(game.get('release_dates')), is_numeric=True)
    result['remakes'] = format_list_for_postgres_array(parse_id_list(game.get('remakes')), is_numeric=True)
    result['remasters'] = format_list_for_postgres_array(parse_id_list(game.get('remasters')), is_numeric=True)
    result['similar_games'] = format_list_for_postgres_array(parse_id_list(game.get('similar_games')), is_numeric=True)
    result['standalone_expansions'] = format_list_for_postgres_array(parse_id_list(game.get('standalone_expansions')), is_numeric=True)
    result['tags'] = format_list_for_postgres_array(parse_id_list(game.get('tags')), is_numeric=True)
    result['expanded_games'] = format_list_for_postgres_array(parse_id_list(game.get('expanded_games')), is_numeric=True)

    # --- Array Fields (Text - simple IDs or complex) ---
    result['dlcs'] = format_list_for_postgres_array(parse_id_list(game.get('dlcs')))
    result['expansions'] = format_list_for_postgres_array(parse_id_list(game.get('expansions')))
    result['ports'] = format_list_for_postgres_array(parse_id_list(game.get('ports')))
    result['age_ratings'] = format_list_for_postgres_array(parse_id_list(game.get('age_ratings')))
    result['alternative_names'] = format_list_for_postgres_array(parse_id_list(game.get('alternative_names')))
    result['language_supports'] = format_list_for_postgres_array(parse_id_list(game.get('language_supports')))
    result['involved_companies'] = format_list_for_postgres_array([]) # Placeholder
    result['psn_np_communication_ids'] = format_list_for_postgres_array([]) # Placeholder


    # --- Array Fields (Text - Slugs from Lookups) ---
    def get_slug_array(column_name, ref_table_key):
        id_list = parse_id_list(game.get(column_name), ref_tables.get(ref_table_key))
        slugs = get_slug_list(id_list)
        return format_list_for_postgres_array(slugs)
    result['collections'] = get_slug_array('collections', 'collections')
    result['game_modes'] = get_slug_array('game_modes', 'game_modes')
    result['genres'] = get_slug_array('genres', 'genres')
    result['keywords'] = get_slug_array('keywords', 'keywords')
    result['platforms'] = get_slug_array('platforms', 'platforms')
    result['player_perspectives'] = get_slug_array('player_perspectives', 'player_perspectives')
    result['themes'] = get_slug_array('themes', 'themes')

    # Developer / Publisher (text[])
    involved_data = ref_tables.get('involved_companies', {}).get(game_id, [])
    developers = set(); publishers = set()
    company_ref = ref_tables.get('companies', {})
    for info in involved_data:
        comp_id = info.get('company'); name = company_ref.get(comp_id)
        if name: name_str = str(name); (developers if info.get('developer') else publishers).add(name_str)
    result['developer'] = format_list_for_postgres_array(sorted(get_slug_list(list(developers))))
    result['publisher'] = format_list_for_postgres_array(sorted(get_slug_list(list(publishers))))


    # --- Single Value Text / Numeric Fields ---
    game_type_id = game.get('game_type'); game_type_name = ref_tables.get('game_types', {}).get(game_type_id); result['game_type'] = slugify(str(game_type_name)) if pd.notna(game_type_name) else ''
    franchise_id = game.get('franchise'); result['franchise'] = ref_tables.get('franchises', {}).get(franchise_id, '') if pd.notna(franchise_id) else ''
    # franchises: text (DDL is single text, join slugs)
    fr_ids = parse_id_list(game.get('franchises'), ref_tables.get('franchises')); result['franchises'] = ','.join(get_slug_list(fr_ids))
    # multiplayer_modes: text (DDL mismatch, format list as text array)
    mp_list = parse_id_list(game.get('multiplayer_modes')); result['multiplayer_modes'] = format_list_for_postgres_array(mp_list)
    pg_id = game.get('parent_game'); result['parent_game'] = int(pg_id) if pd.notna(pg_id) and str(pg_id).isdigit() else pd.NA
    status_id = game.get('status'); status_name = ref_tables.get('game_statuses', {}).get(status_id); result['game_status'] = slugify(str(status_name)) if pd.notna(status_name) else ''
    result['storyline'] = str(game.get('storyline', '')) if pd.notna(game.get('storyline')) else ''
    result['summary'] = str(game.get('summary', '')) if pd.notna(game.get('summary')) else ''
    result['version_title'] = str(game.get('version_title', '')) if pd.notna(game.get('version_title')) else ''

    # Numeric fields
    def get_numeric_or_na(key, target_type=float):
        # ... (Keep existing robust code) ...
        val = game.get(key);
        if pd.isna(val): return pd.NA
        try: return target_type(val)
        except (ValueError, TypeError): return pd.NA
    result['hypes'] = get_numeric_or_na('hypes', int)
    result['aggregated_rating'] = get_numeric_or_na('aggregated_rating', float)
    result['aggregated_rating_count'] = get_numeric_or_na('aggregated_rating_count', int)
    result['rating'] = get_numeric_or_na('rating', float)
    result['rating_count'] = get_numeric_or_na('rating_count', int)
    result['total_rating'] = get_numeric_or_na('total_rating', float)
    result['total_rating_count'] = get_numeric_or_na('total_rating_count', int)


    # --- Placeholder Fields ---
    result['pop_score_visits'] = pd.NA; result['pop_score_want_to_play'] = pd.NA
    result['pop_score_playing'] = pd.NA; result['pop_score_played'] = pd.NA
    result['steam_24hr_peak_players'] = pd.NA; result['steam_positive_reviews'] = pd.NA
    result['steam_negative_reviews'] = pd.NA; result['steam_total_reviews'] = pd.NA
    result['game_version'] = pd.NA

    return result


# %%
# --- Main Dataset Generation Function ---

def generate_games_dataset():
    logger.info("Début génération dataset (Format PostgreSQL Array)...")
    if 'dump_file_paths' not in globals(): logger.critical("'dump_file_paths' non définie."); return pd.DataFrame()
    dump_paths = get_dump_paths();
    if not dump_paths: logger.error("Aucun chemin de dump trouvé."); return pd.DataFrame()
    dfs = load_dumps(dump_paths);
    if 'games' not in dfs or dfs['games'].empty: logger.error("Dump 'games' essentiel manquant/vide."); return pd.DataFrame()
    ref_tables = create_reference_tables(dfs)

    logger.info("Traitement des jeux...")
    games_df = dfs['games']
    if 'id' not in games_df.columns: logger.error("Colonne 'id' manquante dans 'games'."); return pd.DataFrame()
    games_df = games_df.dropna(subset=['id'])
    games_df = games_df.drop_duplicates(subset=['id'], keep='first')

    results = []
    total_games = len(games_df)
    logger.info(f"Début traitement de {total_games} jeux...")
    for i, game in games_df.iterrows():
        if (i + 1) % 1000 == 0: logger.info(f"Traitement jeu {i+1}/{total_games}...")
        processed_data = process_game(game, ref_tables)
        if processed_data: results.append(processed_data)

    if not results: logger.error("Aucun jeu traité avec succès."); return pd.DataFrame()

    logger.info(f"Traitement terminé: {len(results)} jeux.")
    logger.info("Création DataFrame résultat...")
    result_df = pd.DataFrame(results)

    # Define column order based on DDL
    columns_order = [
        'slug', 'title', 'id_igdb', 'steam_app_id', 'psn_app_id', 'xbox_app_id',
        'release_date', 'updated_at', 'game_status', 'game_type', 'cover',
        'artworks', 'screenshots', 'websites', 'developer', 'publisher',
        'genres', 'themes', 'keywords', 'game_modes', 'player_perspectives',
        'platforms', 'collections', 'franchise', 'franchises', 'parent_game',
        'bundles', 'dlcs', 'expansions', 'standalone_expansions', 'remakes',
        'remasters', 'ports', 'expanded_games', 'similar_games', 'summary',
        'storyline', 'rating', 'rating_count', 'total_rating', 'total_rating_count',
        'aggregated_rating', 'aggregated_rating_count', 'hypes', 'age_ratings',
        'alternative_names', 'version_title', 'game_localizations', 'language_supports',
        'multiplayer_modes', 'tags', 'pop_score_visits', 'pop_score_want_to_play',
        'pop_score_playing', 'pop_score_played', 'steam_24hr_peak_players',
        'steam_positive_reviews', 'steam_negative_reviews', 'steam_total_reviews',
        'game_version', 'psn_np_communication_ids' # Keep placeholders, remove involved_companies if not needed
    ]

    # Add missing columns and ensure correct order
    for col in columns_order:
        if col not in result_df.columns:
            logger.warning(f"Colonne '{col}' non générée, ajoutée vide/NA.")
            # Assign default based on DDL guess
            if any(t in col for t in ['date', 'at']): result_df[col] = pd.NaT
            elif any(t in col for t in ['id', 'count', 'hypes', 'version', 'parent', 'reviews', 'players', 'steam_app', 'psn_app']): result_df[col] = pd.NA
            elif any(t in col for t in ['rating', 'score']): result_df[col] = pd.NA
            elif '[]' in str(games_dataset_pg.columns[games_dataset_pg.columns.get_loc(col)].type) if col in games_dataset_pg.columns else True: result_df[col] = '{}' # Crude check for array types in DDL
            else: result_df[col] = ''

    result_df = result_df[columns_order]

    output_filename = 'games_dataset_pg.csv'
    logger.info(f"Enregistrement dataset formaté PG dans {output_filename}...")
    try:
        result_df.to_csv(
    output_filename,
    index=False,
    quoting=csv.QUOTE_MINIMAL, # Quotes all non-numeric fields
    # OR quoting=csv.QUOTE_ALL, # Quotes absolutely everything
    na_rep='',
    encoding='utf-8',
    header=True
)
        logger.info(f"Dataset généré: {len(result_df)} jeux enregistrés dans {output_filename}")
    except Exception as e: logger.error(f"Erreur enregistrement CSV {output_filename}: {e}", exc_info=True)

    return result_df


# %%
# --- Execution ---
try:
    logger.info("Lancement generate_games_dataset() pour format PostgreSQL...")
    games_dataset_pg = generate_games_dataset()
    if not games_dataset_pg.empty:
        logger.info("Aperçu dataset généré (format PG Array):")
        print(games_dataset_pg.head().to_string()) # Use to_string for better console view
    else: logger.warning("Génération a résulté en DataFrame vide.")
except Exception as e:
    logger.critical(f"Erreur majeure exécution generate_games_dataset: {e}", exc_info=True)
    games_dataset_pg = pd.DataFrame()
chime.success()

2025-06-16 13:30:19,740 - INFO - Lancement generate_games_dataset() pour format PostgreSQL...
2025-06-16 13:30:19,741 - INFO - Début génération dataset (Format PostgreSQL Array)...
2025-06-16 13:30:19,741 - INFO - Récupération des chemins des dumps IGDB depuis dump_file_paths...
2025-06-16 13:30:19,742 - INFO - Dump trouvé pour games: igdb_datadumps\1750053600_games.csv
2025-06-16 13:30:19,742 - INFO - Dump trouvé pour genres: igdb_datadumps\1750053600_genres.csv
2025-06-16 13:30:19,743 - INFO - Dump trouvé pour platforms: igdb_datadumps\1750053600_platforms.csv
2025-06-16 13:30:19,744 - INFO - Dump trouvé pour themes: igdb_datadumps\1750053600_themes.csv
2025-06-16 13:30:19,744 - INFO - Dump trouvé pour keywords: igdb_datadumps\1750053600_keywords.csv
2025-06-16 13:30:19,745 - INFO - Dump trouvé pour game_modes: igdb_datadumps\1750053600_game_modes.csv
2025-06-16 13:30:19,745 - INFO - Dump trouvé pour player_perspectives: igdb_datadumps\1750053600_player_perspectives.csv
2025-06-16 13

                           slug                          title  id_igdb steam_app_id psn_app_id xbox_app_id              release_date                       updated_at game_status  game_type                                                                                cover                                                                   artworks                                                                                                                                                                                                                                                                                                                                                                screenshots                                                                                                       websites           developer publisher                         genres      themes                                                                                                  keywor

In [3]:
pd.set_option('display.max_columns', None)
games_dataset_pg

,slug,title,id_igdb,steam_app_id,psn_app_id,xbox_app_id,release_date,updated_at,game_status,game_type,cover,artworks,screenshots,websites,developer,publisher,genres,themes,keywords,game_modes,player_perspectives,platforms,collections,franchise,franchises,parent_game,bundles,dlcs,expansions,standalone_expansions,remakes,remasters,ports,expanded_games,similar_games,summary,storyline,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count,hypes,age_ratings,alternative_names,version_title,game_localizations,language_supports,multiplayer_modes,tags,pop_score_visits,pop_score_want_to_play,pop_score_playing,pop_score_played,steam_24hr_peak_players,steam_positive_reviews,steam_negative_reviews,steam_total_reviews,game_version,psn_np_communication_ids
0,nightmare-kart-the-old-karts,Nightmare Kart: The Old Karts,330684,<NA>,<NA>,,NaT,2025-06-16 11:30:56.429520+00:00,,update,https://images.igdb.com/igdb/image/upload/t_sc...,{},"{""https://images.igdb.com/igdb/image/upload/t_...",{},"{""lwmedia""}",{},"{""arcade"",""racing""}","{""action""}",{},"{""single-player"",""multiplayer"",""split-screen""}","{""third-person""}","{""pc-microsoft-windows""}",{},,,<NA>,{},{},{},{},{},{},{},{},{},An upcoming free expansion to Nightmare Kart w...,,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,{},{},,{},{},{},{},<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,{}
1,the-undying-beast,The Undying Beast,177310,<NA>,<NA>,,2020-04-13 00:00:00+00:00,2025-06-16 11:30:56.434518+00:00,,main-game,https://images.igdb.com/igdb/image/upload/t_sc...,"{""https://images.igdb.com/igdb/image/upload/t_...","{""https://images.igdb.com/igdb/image/upload/t_...","{""https://www.twitch.tv/directory/game/The%20U...","{""kira""}",{},"{""indie"",""simulator""}","{""horror""}",{},"{""single-player""}","{""first-person""}","{""pc-microsoft-windows""}",{},,,<NA>,{},{},{},{},{},{},{},{},"{65827,29348,13189,55590,87975,116582,36269,55...","There was a flash of light, a choir of glass, ...",,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,{},{},,{},{},{},"{268435488,19,268435469}",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,{}
2,sengoku-anthology,Sengoku Anthology,43460,<NA>,<NA>,,2009-09-01 00:00:00+00:00,2025-06-16 11:30:56.435520+00:00,,bundle,https://images.igdb.com/igdb/image/upload/t_sc...,{},{},{},{},{},"{""hack-and-slash-beat-em-up""}","{""action""}","{""compilation"",""martial-arts"",""brawler""}",{},{},"{""playstation-2""}","{""sengoku""}",,,<NA>,{},{},{},{},{},{},{},{},"{103168,113895,5867,14704,115283,113109,54775,...",Sengoku Anthology is a collection of the three...,,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"{""15301""}",{},,{},{},{},"{536871872,1,536871894,268435481,536870973}",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,{}
3,rockman-x-new-year-2023,Rockman X: New Year 2023,282079,<NA>,<NA>,,2023-01-01 00:00:00+00:00,2025-06-16 11:30:56.436518+00:00,,mod,https://images.igdb.com/igdb/image/upload/t_sc...,{},{},"{""https://borokobo.web.fc2.com/""}","{""puresabe""}",{},"{""platform"",""shooter""}","{""action""}","{""unofficial""}","{""single-player""}","{""third-person"",""side-view""}","{""super-famicom""}","{""rockman-x-new-year""}",,,<NA>,{},{},{},{},{},{},{},{},"{57187,32902,43367,103292,119177,95340,38030,2...",A small hack to celebrate the Chinese New Year...,,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,{},{},,{},"{""876934""}",{},"{268435464,1,536872916,268435461}",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,{}
4,ace-wo-nerae,Ace wo Nerae!,63844,<NA>,<NA>,,1993-12-22 00:00:00+00:00,2025-06-16 11:30:56.436518+00:00,,main-game,https://images.igdb.com/igdb/image/upload/t_sc...,{},"{""https://images.igdb.com/igdb/image/upload/t_...","{""https://en.wikipedia.org/wiki/Aim_for_the_Ac...","{""nippon-telenet""}",{},"{""sport""}","{""action""}","{""snes-enhancement-chip-dsp-1"",""based-on-anime...","{""single-player"",""multiplayer"",""split-screen""}","{""third-person""}","{""super-famicom""}","{""ace-wo-nerae""}",,aim-for-the-ace,<NA>,{},{},{},{},{},{},{},{},"{61985,61127,2442

In [4]:
# %% [markdown]
# # Cellule 5: Connexion et Importation via Lecture CSV + execute_values (CORRIGÉ v5 - Filtre slug NULL/Vide)

# %%
import psycopg2
import psycopg2.extras
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
import re
import time
from tqdm import tqdm

# --- Charger les variables d'environnement ---
# ... (idem) ...
load_dotenv()
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")

# --- Configuration Fichier et Table ---
# ... (idem) ...
csv_file_path = 'games_dataset_pg.csv'
target_table_name = "games"
conflict_column = "id_igdb"

# --- Colonnes attendues ---
# ... (idem) ...
columns_order = [
    'slug', 'title', 'id_igdb', 'steam_app_id', 'psn_app_id', 'xbox_app_id',
    'release_date', 'updated_at', 'game_status', 'game_type', 'cover',
    'artworks', 'screenshots', 'websites', 'developer', 'publisher',
    'genres', 'themes', 'keywords', 'game_modes', 'player_perspectives',
    'platforms', 'collections', 'franchise', 'franchises', 'parent_game',
    'bundles', 'dlcs', 'expansions', 'standalone_expansions', 'remakes',
    'remasters', 'ports', 'expanded_games', 'similar_games', 'summary',
    'storyline', 'rating', 'rating_count', 'total_rating', 'total_rating_count',
    'aggregated_rating', 'aggregated_rating_count', 'hypes', 'age_ratings',
    'alternative_names', 'version_title', 'game_localizations', 'language_supports',
    'multiplayer_modes', 'tags', 'pop_score_visits', 'pop_score_want_to_play',
    'pop_score_playing', 'pop_score_played', 'steam_24hr_peak_players',
    'steam_positive_reviews', 'steam_negative_reviews', 'steam_total_reviews',
    'game_version', 'psn_np_communication_ids'
]

# --- Identifier les colonnes Array ---
# ... (idem) ...
pg_array_cols = [
    'artworks', 'bundles', 'collections', 'dlcs', 'expansions',
    'game_localizations', 'game_modes', 'genres', 'involved_companies',
    'keywords', 'language_supports', 'platforms', 'player_perspectives',
    'ports', 'age_ratings', 'alternative_names', 'release_dates',
    'remakes', 'remasters', 'screenshots', 'similar_games',
    'standalone_expansions', 'tags', 'themes', 'websites',
    'developer', 'publisher', 'expanded_games', 'psn_np_communication_ids'
]
pg_int_array_cols = [
    'bundles', 'dlcs', 'expansions', 'game_localizations', 'language_supports',
    'ports', 'release_dates', 'remakes', 'remasters', 'similar_games',
    'standalone_expansions', 'tags', 'expanded_games'
]

# --- Fonction de parsing ---
# ... (idem) ...
PG_ARRAY_REGEX = re.compile(r'"((?:[^"\\]|\\.)*)"|([^",{} ]+)')
def parse_pg_array_string(array_string):
    # ... (idem) ...
    if pd.isna(array_string) or not isinstance(array_string, str) or array_string.strip() == '': return None
    array_string = array_string.strip()
    if array_string == '{}': return []
    if not (array_string.startswith('{') and array_string.endswith('}')): return None
    content = array_string[1:-1].strip();
    if not content: return []
    elements = [];
    try:
        for match in PG_ARRAY_REGEX.finditer(content):
            quoted, unquoted = match.groups()
            if quoted is not None: elements.append(quoted.replace('\\"', '"').replace('\\\\', '\\'))
            elif unquoted is not None: elements.append(None if unquoted.strip() == 'NULL' else unquoted.strip())
        return elements
    except Exception: return None

# --- Initialisation ---
# ... (idem) ...
conn = None; cur = None; total_read = 0; total_prepared = 0; total_processed_db = 0
total_skipped_null_slug_str = 0 # Pour le filtre 'null' string
total_skipped_null_slug_val = 0 # Pour le filtre NULL/vide final
can_proceed = True

# --- Valider Variables Env & Fichier CSV ---
# ... (idem) ...
missing_vars = [var for var in ['DB_NAME', 'DB_USER', 'DB_PASSWORD', 'DB_HOST', 'DB_PORT'] if not os.getenv(var)]
if missing_vars: print(f"ERREUR: Env vars manquantes : {', '.join(missing_vars)}"); can_proceed = False
if not os.path.exists(csv_file_path): print(f"ERREUR: Fichier CSV '{csv_file_path}' introuvable."); can_proceed = False

# --- Traitement ---
if can_proceed:
    df_import = None; data_tuples = []

    # --- 1. Lire le fichier CSV ---
    # ... (idem) ...
    try:
        print(f"\nLecture du fichier CSV '{csv_file_path}'...")
        df_import = pd.read_csv(csv_file_path, keep_default_na=True, na_values=[''], dtype=str)
        total_read = len(df_import)
        print(f"CSV lu. {total_read} lignes trouvées.")
        df_import.replace({np.nan: None, '<NA>': None, 'NaT': None}, inplace=True)
    except Exception as e: print(f"Erreur lecture CSV : {e}"); import traceback; traceback.print_exc(); can_proceed = False


    # --- 2. Préparer les données (Avec DEUX Filtres Slug et Conversion Type Array Int) ---
    if can_proceed and df_import is not None:
        print("\nPréparation des données pour l'insertion (parsing, conversion, filtre)...")
        try:
            df_import = df_import[columns_order]
            slug_col_index = columns_order.index('slug')
            id_col_index = columns_order.index('id_igdb') # Aussi utile pour validation
        except (KeyError, ValueError) as e:
             print(f"ERREUR CRITIQUE : Colonne CSV essentielle ('slug' ou 'id_igdb') manquante ou problème d'index : {e}.")
             can_proceed = False

        if can_proceed:
            prepared_rows = []
            for row_tuple in tqdm(df_import.itertuples(index=False, name=None), total=total_read, desc="Préparation lignes"):
                processed_row = list(row_tuple); valid_row = True

                # Filtre 1: Ignorer les lignes où le slug est la chaîne 'null'
                slug_value_initial = processed_row[slug_col_index]
                if isinstance(slug_value_initial, str) and slug_value_initial == 'null':
                    total_skipped_null_slug_str += 1
                    continue # Passer à la ligne suivante

                # Boucle de processing des colonnes
                for i, col_name in enumerate(columns_order):
                    value = processed_row[i]
                    # A. Traiter Colonnes Array
                    if col_name in pg_array_cols and isinstance(value, str):
                        parsed_array = parse_pg_array_string(value)
                        if col_name in pg_int_array_cols and isinstance(parsed_array, list):
                            converted_int_array = []
                            conversion_ok = True
                            for item_str in parsed_array:
                                if item_str is None: converted_int_array.append(None)
                                else:
                                    try: converted_int_array.append(int(item_str))
                                    except (ValueError, TypeError): conversion_ok = False; break
                            processed_row[i] = converted_int_array if conversion_ok else None
                        else: processed_row[i] = parsed_array
                    # B. Traiter Valeurs Manquantes (non-array)
                    elif pd.isna(value) or value is None: # Ajout value is None
                        processed_row[i] = None
                    # C. Traiter Scalaires (Conversions)
                    elif col_name in ['steam_app_id', 'psn_app_id', 'hypes', 'aggregated_rating_count', 'rating_count', 'total_rating_count', 'parent_game', 'steam_24hr_peak_players', 'steam_positive_reviews', 'steam_negative_reviews', 'steam_total_reviews', 'game_version', 'id_igdb']:
                         try: processed_row[i] = int(value) if value is not None else None
                         except (ValueError, TypeError): processed_row[i] = None
                    elif col_name in ['aggregated_rating', 'rating', 'total_rating', 'pop_score_visits', 'pop_score_want_to_play', 'pop_score_playing', 'pop_score_played']:
                          try: processed_row[i] = float(value) if value is not None else None
                          except (ValueError, TypeError): processed_row[i] = None
                    elif col_name in ['release_date', 'updated_at']:
                         try:
                             dt_obj = pd.to_datetime(value, errors='coerce', utc=True)
                             processed_row[i] = dt_obj.to_pydatetime() if pd.notna(dt_obj) else None
                         except Exception: processed_row[i] = None
                    # else: la valeur string reste (déjà traitée par read_csv dtype=str et replace NA->None)

                    # D. Validation Critique id_igdb (redondant car clé primaire mais bon)
                    if col_name == 'id_igdb' and processed_row[i] is None:
                         # print(f"WARN: id_igdb NULL Ligne ignorée.")
                         valid_row = False; break


                # **** AJOUT FILTRE FINAL : Vérifier si SLUG ou ID sont None/Vide APRES processing ****
                if valid_row:
                    final_slug = processed_row[slug_col_index]
                    final_id = processed_row[id_col_index]

                    # Vérifier slug (NOT NULL): Ne doit pas être None ou chaîne vide
                    if final_slug is None or (isinstance(final_slug, str) and not final_slug.strip()):
                        total_skipped_null_slug_val += 1
                        valid_row = False
                        # print(f"WARN: Slug final NULL ou vide pour ID {final_id}. Ligne ignorée.")

                    # Vérifier id_igdb (PK, NOT NULL): Ne doit pas être None
                    # (Normalement déjà couvert par la validation précédente)
                    elif final_id is None:
                         valid_row = False
                         # print(f"WARN: ID IGDB final NULL. Ligne ignorée.")

                # Ajouter seulement si toutes les validations sont passées
                if valid_row:
                    prepared_rows.append(tuple(processed_row))

            data_tuples = prepared_rows; total_prepared = len(data_tuples)
            print(f"Préparation terminée. {total_prepared} lignes prêtes ({total_skipped_null_slug_str} lignes avec slug='null' string ignorées, {total_skipped_null_slug_val} lignes avec slug final NULL/vide ignorées).")

    # --- 3. Insérer/Mettre à jour en base de données ---
    if can_proceed and data_tuples:
        try:
            print("\nTentative connexion DB...")
            # ... (connexion idem) ...
            conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT)
            conn.set_client_encoding('UTF8'); cur = conn.cursor(); print("Connexion réussie !")

            # --- Requête SQL (idem) ---
            cols_sql = ", ".join([f'"{col}"' for col in columns_order])
            sql_values_placeholder = "%s"
            update_columns = [col for col in columns_order if col != conflict_column]
            update_set_sql = ", ".join([f'"{col}" = EXCLUDED."{col}"' for col in update_columns])
            sql_upsert = f"""
                INSERT INTO "{target_table_name}" ({cols_sql})
                VALUES {sql_values_placeholder}
                ON CONFLICT ("{conflict_column}") DO UPDATE
                SET {update_set_sql};
            """
            print("\nRequête UPSERT préparée...")

            print(f"\nExécution UPSERT pour {total_prepared} lignes...")
            start_db = time.time()

            # --- execute_values (idem) ---
            psycopg2.extras.execute_values(
                cur, sql_upsert, data_tuples, template=None, page_size=500
            )

            total_processed_db = cur.rowcount if cur.rowcount >= 0 else total_prepared
            db_time = time.time() - start_db
            print(f"Opération DB terminée en {db_time:.2f} s. {total_processed_db} lignes traitées.")

            conn.commit(); print("Transaction validée.")

        except (Exception, psycopg2.Error) as error:
            # ... (error handling idem) ...
            print(f"\nERREUR opération DB : {error}")
            if hasattr(error, 'pgcode'): print(f"Code PG: {error.pgcode}")
            if hasattr(error, 'pgerror'): print(f"Msg PG: {error.pgerror}")
            # Imprimer la ligne fautive si psycopg2 la fournit (pas toujours le cas pour NotNullViolation)
            if hasattr(error, 'diag') and error.diag.message_detail: print(f"Detail: {error.diag.message_detail}")
            import traceback; traceback.print_exc()
            if conn: 
                try: 
                    print("Rollback..."); conn.rollback(); 
                except Exception as rb_e: print(f"Erreur rollback: {rb_e}")
        finally:
            # ... (finally block idem) ...
            if cur: cur.close()
            if conn: conn.close(); print("Connexion PostgreSQL fermée.")

# ... (Reste du script / Résumé Final) ...
print(f"\n--- Résumé ---")
print(f"Lignes lues du CSV: {total_read}")
print(f"Lignes avec slug='null' string ignorées: {total_skipped_null_slug_str}")
print(f"Lignes avec slug final NULL/vide ignorées: {total_skipped_null_slug_val}")
print(f"Lignes préparées (après parsing/validation/filtres): {total_prepared}")
print(f"Lignes traitées par la base de données (estimation): {total_processed_db}")
chime.success()


Lecture du fichier CSV 'games_dataset_pg.csv'...
CSV lu. 319994 lignes trouvées.

Préparation des données pour l'insertion (parsing, conversion, filtre)...


Préparation lignes: 100%|██████████| 319994/319994 [04:04<00:00, 1309.89it/s]


Préparation terminée. 319993 lignes prêtes (0 lignes avec slug='null' string ignorées, 1 lignes avec slug final NULL/vide ignorées).

Tentative connexion DB...
Connexion réussie !

Requête UPSERT préparée...

Exécution UPSERT pour 319993 lignes...
Opération DB terminée en 330.61 s. 493 lignes traitées.
Transaction validée.
Connexion PostgreSQL fermée.

--- Résumé ---
Lignes lues du CSV: 319994
Lignes avec slug='null' string ignorées: 0
Lignes avec slug final NULL/vide ignorées: 1
Lignes préparées (après parsing/validation/filtres): 319993
Lignes traitées par la base de données (estimation): 493
